In [0]:
#IMPORTANTE:

#Importamos la librería para generar mensajes de log en la ejecución de la función
import logging

#Importamos la librería de azure functions para implementar funciones
#import azure.functions as func

#Importamos la librería para leer archivos desde una CUENTA DE ALMACENAMIENTO
from azure.storage.blob import BlobServiceClient

#Librería para conversión de STRINGs a BYTEs
import io

#Librería para manipulación de archivos en el SISTEMA OPERATIVO
import os

#Utilitario para iniciar sesión en el servicio
from azure.ai.translation.text import TranslatorCredential

#Utilitario para conectarnos al servicio
from azure.ai.translation.text import TextTranslationClient

#Utilitario para enviar el texto por traducir
from azure.ai.translation.text.models import InputTextItem

#Respuesta que devolverá la función
#Inicialmente está vacía
respuesta = None

#Obtenemos la cadena de conexión a la CUENTA DE ALMACENAMIENTO de la función
#connectionString = os.environ["AzureWebJobsStorage"]

#Obtenemos un cliente de manipulación de la CUENTA DE ALMACENAMIENTO de la función
#storageAccountClient = BlobServiceClient.from_connection_string(connectionString)

#Nos conectamos a un contenedor del BLOB STORAGE
#IMPORTANTE: CAMBIAR "XXX" POR NUESTRAS INICIALES
#blobStorageClient = storageAccountClient.get_container_client("datasetsbda")

#Nos conectamos al archivo INPUT del BLOB STORAGE
#blobInput = blobStorageClient.get_blob_client("/data/documento.txt")

#Descargamos el archivo en una variable
#archivoInput = blobInput.download_blob()

#Leemos el contenido del archivo
#contenidoInput = archivoInput.readall().decode("utf-8")

#Documentos que se procesan
documentos = [{"text":"Padre nuestro que estas en los cielos"},{"text": "santificado sea tu nombre"}]

#Agregamos el documento leido
#documentos.append(contenidoInput)

#Región donde se desplegó el servicio
region = "eastus"

#Clave de acceso al servicio
clave = "75ee518d817e4256a16e74117637d750"

#Definimos las credenciales de acceso
credenciales = TranslatorCredential(clave, region)

#Iniciamos sesión sobre el servicio
servicio = TextTranslationClient(
    credential = credenciales
)

#Definimos los lenguajes a los que se traducirán los textos
lenguajes = ["en"]

#Definimos el array de documentos que se enviarán
documentosPorTraducir = []

#Colocamos los documentos
for documento in documentos:
    documentosPorTraducir.append(
        InputTextItem(text = documento)
    )

#Realizamos la consulta enviando los documentos y definiendo los lenguajes a los que los traducimos
resultados = servicio.translate(
    content = documentosPorTraducir, 
    to = lenguajes
)

#Acumulamos el analisis de cada documento
descripcion = ""

#Iteramos cada resultado de cada documento
for resultado in resultados:

    #Iteramos cada traducción
    for traduccion in resultado["translations"]:

        linea = (
            str(resultado["detectedLanguage"]["language"]) + "|" +
            str(resultado["detectedLanguage"]["score"]) + "|" +
            str(traduccion["to"]) + "|" +
            str(traduccion["text"])
        )

        #Agregamos cada línea a la descripción
        descripcion =  descripcion + linea + "\n"

#Le agregamos una cabecera
descripcion = "IDIOMA_ORIGEN|PROBABILIDAD|IDIOMA_DESTINO|TRADUCCION" + "\n" + descripcion

#Para escribir el resultado en un archivo sobre un blob storage debemos binarizarlo
#Binarizamos el resultado
contenidoOutputBin = descripcion.encode("utf-8")

#Nos conectamos al archivo OUTPUT del BLOB STORAGE
blobOutput = blobStorageClient.get_blob_client("/respuesta_desde_azure_function.txt")

#Escribimos el resultado
blobOutput.upload_blob(contenidoOutputBin, overwrite=True)

#Colocamos el resultado en la respuesta para visualizarlo
respuesta = func.HttpResponse(descripcion)

#Retornamos la respuesta
return respuesta

---------------------------------------------------------------------------
HttpResponseError                         Traceback (most recent call last)
File <command-3108624137709860>:83
     78     documentosPorTraducir.append(
     79         InputTextItem(text = documento)
     80     )
     82 #Realizamos la consulta enviando los documentos y definiendo los lenguajes a los que los traducimos
---> 83 resultados = servicio.translate(
     84     content = documentosPorTraducir, 
     85     to = lenguajes
     86 )
     88 #Acumulamos el analisis de cada documento
     89 descripcion = ""

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/azure/core/tracing/decorator.py:78, in distributed_trace.<locals>.decorator.<locals>.wrapper_use_tracer(*args, **kwargs)
     76 span_impl_type = settings.tracing_implementation()
     77 if span_impl_type is None:
---> 78     return func(*args, **kwargs)
     80 # Merge span is parameter is set, but only if no ex